## User Story: Load Data

Loading your user-collections is a simple process, in order to ensure this notebook works, make sure to already have results of a job saved
in your workspace so we can load it here.
If you don't you can check out the save result demo notebook first.

In [1]:
import openeo

# Connect to the openEO backend and authenticate with EGI Check-In

connection = openeo.connect("https://openeo.eodc.eu/openeo/1.1.0")
connection = connection.authenticate_oidc(provider_id="egi")

Authenticated using refresh token.


In order to load a user collection, we have to first find out the collection id, we can do this by listing the stac collections within the workspace using
the list_stac_collections(WORKSPACE_NAME) function. This function returns us a list of tuples consisting of the collection_id and the path to the stac collection json.

In [2]:
from eodc.workspace import CephAdapter, EODC_CEPH_URL

# Set these variables to your own.

WORKSPACE_NAME = "testing"

S3_ENDPOINT = EODC_CEPH_URL
S3_ACCESS_KEY = "KQXVUVGIJU33L85J7SW9"
S3_SECRET_KEY = "fLvbLbJMoYjeztdsYybfF25NqnDz8gaE0XmZFqfq"

adapter: CephAdapter = CephAdapter(S3_ENDPOINT, S3_ACCESS_KEY, S3_SECRET_KEY)

collections = adapter.list_stac_collections(WORKSPACE_NAME)

collections

[('openeo_117b06ce-9d48-4c4e-89e1-7f6c48821e15',
  <Collection id=openeo_117b06ce-9d48-4c4e-89e1-7f6c48821e15>),
 ('openeo_4ec93aba-0c47-41c9-b28f-123189caa4d9',
  <Collection id=openeo_4ec93aba-0c47-41c9-b28f-123189caa4d9>),
 ('openeo_5712a088-c30e-4563-9fde-0a91af9475a1',
  <Collection id=openeo_5712a088-c30e-4563-9fde-0a91af9475a1>),
 ('openeo_6b23895b-16e5-4769-af85-8e06d1aef961',
  <Collection id=openeo_6b23895b-16e5-4769-af85-8e06d1aef961>),
 ('openeo_958d2044-0d83-4daf-a193-56983ab6f9ad',
  <Collection id=openeo_958d2044-0d83-4daf-a193-56983ab6f9ad>),
 ('openeo_af7e4a77-4504-4910-9307-eed5ef045226', <Collection id=renamedid>),
 ('openeo_d7c410a9-9b3e-431f-a808-e2e54bf7795b', <Collection id=test>),
 ('renamedid', <Collection id=renamedid>)]

Now you can set you url manually or get it via the list_stac_collections return value.

In [ ]:
# URL = ""

collection_id = collections[-1][0]

collection = collections[-1][-1]

URL = collection.links[1].absolute_href

URL

'/vsis3/testing/renamedid/STAC/renamedid_collection.json'

You can also check the available STAC items of your given collection beforehand by using the get_stac_items function of the workspace adapter.

In [20]:
items = adapter.get_stac_items(workspace_name=WORKSPACE_NAME, collection_id=collection_id)

items

[<Item id=EU010M_E052N016T1_20190121T120000>,
 <Item id=EU010M_E052N016T1_20190111T120000>]

We are also defaulting to the same spatial_extent and the same temporal_extent as in the save result notebook, feel free to change these to fit your use case.

If you want to check the full extent of your user collection you can use the following code:

In [21]:
print(adapter.get_collection(WORKSPACE_NAME, collection_id).extent.spatial.bboxes) # This prints the maximum bounding box around all items in the STAC collection
print(adapter.get_collection(WORKSPACE_NAME, collection_id).extent.temporal.intervals) # This prints the maximum temporal extent of all items in the STAC collection

[[16.115783, 48.054209, 16.628694, 48.376627]]
[[datetime.datetime(2019, 1, 11, 12, 0, tzinfo=tzutc()), datetime.datetime(2019, 1, 21, 12, 0, tzinfo=tzutc())]]


Now we can start a job with the corresponding collection_id in the load_collection node, with the workspace property filled out.

This will load results from a previous job which we can then further transform and use as we like.

In [ ]:
collection = connection.load_stac(
    url=URL,
    spatial_extent={
        "west": 16.156771491786476,
        "east": 16.59018048465475,
        "south": 48.08419286799747,
        "north": 48.34670064966687,
    },
    temporal_extent=["2019-01-01T00:00:00Z", "2019-01-31T00:00:00Z"],
    properties={}
)

result = collection.save_result(format="netCDF")

Run this to start the job.

In [ ]:
job = connection.create_job()

job.start()

Run this to check the status of your job.

In [ ]:
job # Execute this to check on your jobs progress

Once the job has run through you can check the results!